In [ ]:
import bpy
import bmesh

# 対象のオブジェクトを指定
obj_name = "Cube"  # ここにオブジェクトの名前を指定
obj = bpy.data.objects[obj_name]
# メッシュデータにアクセス
mesh = obj.data
# 頂点カラーレイヤーが存在するか確認
if not mesh.vertex_colors:
    print("The object does not have vertex colors.")
else:
    # BMeshオブジェクトの作成とメッシュデータのロード
    bm = bmesh.new()
    bm.from_mesh(mesh)
    # 頂点カラーレイヤーの取得
    color_layer = bm.loops.layers.color.active
    # 出力結果を保持するリスト
    result = []
    # 各面に対して処理
    for face in bm.faces:
        # 各ループ（頂点とそれに接続されるエッジのセット）に対して処理
        for loop in face.loops:
            # 頂点座標の取得
            co = loop.vert.co
            # 頂点カラーの取得
            color = loop[color_layer]
            # 結果をリストに追加
            result.append([co.x, co.y, co.z, color[0], color[1], color[2]])
    # BMeshのクリア
    bm.free()
    # テキストファイルへの出力
    with open("C:/Users/ku-admin/Desktop/output.txt","w") as f:
        for r in result:
            f.write(", ".join(map(str, r)) + "\n")

In [1]:
import bpy

# 選択されたオブジェクトを取得
obj = bpy.context.object
# マテリアルが存在するか確認
if obj.data.materials:
    # 最初のマテリアルを取得
    material = obj.data.materials[0]
    # マテリアルにテクスチャが設定されているか確認
    if material.node_tree:
        # テクスチャノードを検索
        texture_node = None
        for node in material.node_tree.nodes:
            if node.type == 'TEX_IMAGE':
                texture_node = node
                break
        if texture_node and texture_node.image:
            image = texture_node.image
            # イメージからピクセルデータを取得
            pixel_data = list(image.pixels)
            # 頂点カラーを設定するための頂点カラーレイヤーを作成
            vcol_layer = obj.data.vertex_colors.new()
            # オブジェクトの頂点をループ処理
            for face in obj.data.polygons:
                for vert_index in face.vertices:
                    # 頂点の頂点カラーを設定
                    vertex_color = vcol_layer.data[vert_index]
                    # テクスチャ座標を計算
                    tex_coords = obj.data.uv_layers.active.data[vert_index].uv
                    tex_x = int(tex_coords[0] * image.size[0])
                    tex_y = int(tex_coords[1] * image.size[1])
                    # ピクセルデータからRGB値を取得
                    pixel_index = (tex_y * image.size[0] + tex_x) * 4
                    r = pixel_data[pixel_index]
                    g = pixel_data[pixel_index + 1]
                    b = pixel_data[pixel_index + 2]
                    # 頂点カラーを設定
                    vertex_color.color = (r, g, b, 1.0)

ModuleNotFoundError: No module named 'bpy'

In [ ]:
import bpy
import bmesh
import numpy as np
from PIL import Image

# 使用するオブジェクトとテクスチャを指定します
object_name = "10200_Pineapple_v1-L2"  # ここにオブジェクトの名前を指定してください
texture_file_path = "C:/Users/ku-admin/Downloads/Pineapple_v1_L2.123cd933e306-5311-404e-bac7-c360b305d8c5/Pineapple_v1_L2.123cd933e306-5311-404e-bac7-c360b305d8c5/10200_Pineapple.jpg"  # ここにテクスチャのファイルパスを指定してください
# 指定されたオブジェクトを取得します
obj = bpy.data.objects[object_name]
mesh = obj.data
# メッシュをbmeshオブジェクトとしてロードします
bm = bmesh.new()
bm.from_mesh(mesh)
# UVレイヤーを取得します
uv_layer = bm.loops.layers.uv.active
# テクスチャイメージをロードします
texture_image = Image.open(texture_file_path)
texture_data = np.array(texture_image)
# 結果を保存するための辞書を作成します
vertex_data = {}
# 各面の各頂点に対してループします
for face in bm.faces:
    for loop in face.loops:
        vertex = loop.vert
        uv_coordinate = loop[uv_layer].uv
        # UV座標をイメージ座標に変換します
        ix, iy = (uv_coordinate * texture_data.shape[:2]).astype(int)
        # テクスチャの色情報を取得します
        color = texture_data[iy, ix]
        # 頂点の座標を取得します
        coordinate = vertex.co
        # 頂点IDをキーとして色情報と座標を保存します
        vertex_data[vertex.index] = {"color": color.tolist(), "coordinate": coordinate[:].tolist()}
# txtファイルに出力します
with open('C:/Users/ku-admin/Desktop/output2.txt', 'w') as f:
    for vertex_index, data in vertex_data.items():
        f.write(f"Vertex ID: {vertex_index}\n")
        f.write(f"Color: {data['color']}\n")
        f.write(f"Coordinate: {data['coordinate']}\n")
        f.write("\n")
print("The script has successfully executed.")

In [ ]:
import bpy
import os

# アクティブなオブジェクトを取得します
obj = bpy.context.active_object
if obj is not None and obj.type == 'MESH':
    # 頂点カラーを取得します
    color_layer = obj.data.vertex_colors.active
    # ユーザーのデスクトップへのパスを設定します
    # Windowsの場合は以下のようになります。{ユーザー名}はあなたのユーザー名に置き換えてください。
    desktop = "C:\\Users\\ku-admin\\Desktop"
    output_file_path = os.path.join(desktop, "output2.txt")
    with open(output_file_path, "w") as file:
        for poly in obj.data.polygons:
            for loop_index in poly.loop_indices:
                loop = obj.data.loops[loop_index]
                # 頂点の座標を出力します
                vert_coord = obj.data.vertices[loop.vertex_index].co
                file.write(f"Vertex Coord: {vert_coord}\n")
                if color_layer is not None:
                    # 頂点カラーを出力します
                    color = color_layer.data[loop_index].color
                    file.write(f"Vertex Color: {color}\n")
else:
    print("Please select a mesh object.")

In [ ]:
import bpy
import os

# 出力ファイルのパスを設定します
output_file_path = os.path.join("C:\\Users\\ku-admin\\Desktop", "output.txt")
# 選択したオブジェクトを取得します
obj = bpy.context.active_object
if obj.type == 'MESH':
    mesh = obj.data
    if len(mesh.vertex_colors) > 0: 
        # 頂点カラーレイヤーがある場合
        # ファイル書き込み用にファイルを開きます
        with open(output_file_path, 'w') as file:
            # 全ての頂点について、座標とカラーを書き出します
            for poly in mesh.polygons:
                for idx in poly.loop_indices:
                    loop = mesh.loops[idx]
                    vertex = mesh.vertices[loop.vertex_index]
                    color = mesh.vertex_colors.active.data[idx].color
                    # ファイルに書き込みます
                    file.write(f"Vertex ID: {loop.vertex_index}, Position: {vertex.co}, Color: {color}\n")
    else:
        # 頂点カラーレイヤーがない場合
        print("The object has no vertex color layer.")
else:
    print("The selected object is not a mesh.")


In [ ]:
import bpy
import os

# 出力ファイルのパスを設定します
output_file_path = os.path.join("C:\\Users\\ku-admin\\Desktop", "output.txt")
# 選択したオブジェクトを取得します
obj = bpy.context.active_object
if obj.type == 'MESH':
    mesh = obj.data
    if len(mesh.vertex_colors) > 0: 
        # 頂点カラーレイヤーがある場合
        # ファイル書き込み用にファイルを開きます
        with open(output_file_path, 'w') as file:
            # 全ての頂点について、座標とカラーを書き出します
            for poly in mesh.polygons:
                for idx in poly.loop_indices:
                    loop = mesh.loops[idx]
                    vertex = mesh.vertices[loop.vertex_index]
                    color = mesh.vertex_colors.active.data[idx].color
                    # 頂点カラーはRGBA形式で取得されるため、それぞれの成分にアクセスします
                    r = color[0]
                    g = color[1]
                    b = color[2]
                    a = color[3]
                    # ファイルに書き込みます
                    file.write(f"Vertex ID: {loop.vertex_index}, Position: {vertex.co}, Color: (R:{r}, G:{g}, B:{b}, A:{a})\n")
    else:
        # 頂点カラーレイヤーがない場合
        print("The object has no vertex color layer.")
else:
    print("The selected object is not a mesh.")


In [ ]:
import bpy

# 選択したオブジェクトの参照を取得
selected_objects = bpy.context.selected_objects
if selected_objects:
    # 出力ファイルのパスと名前
    output_file = "output.txt"
    # ファイルを開いて書き込みモードで準備
    with open(output_file, "w") as file:
        for obj in selected_objects:
            if obj.type == "MESH":
                mesh = obj.data
                vertices = mesh.vertices
                vertex_colors = mesh.vertex_colors.get("Col")  # 頂点カラーレイヤーの名前を指定
                # 頂点情報をファイルに書き込む
                for vertex in vertices:
                    coord = obj.matrix_world @ vertex.co  # グローバル座標系での頂点座標
                    line = f"Vertex: {coord}, Color: "
                    if vertex_colors:
                        color = vertex_colors.data[vertex.index].color
                        line += f"({color.r}, {color.g}, {color.b})"
                    else:
                        line += "(No Color)"
                    file.write(line + "\n")
    print(f"頂点情報を {output_file} に出力しました。")
else:
    print("オブジェクトが選択されていません。")


In [ ]:
import bpy
#これ使える　いらないvertex colorみたいな文字を消せば   
# 選択したオブジェクトの参照を取得
selected_objects = bpy.context.selected_objects
if selected_objects:
    # 出力ファイルのパスと名前
    output_file = "output.txt"
    # ファイルを開いて書き込みモードで準備
    with open(output_file, "w") as file:
        for obj in selected_objects:
            if obj.type == "MESH":
                mesh = obj.data
                vertices = mesh.vertices
                vertex_colors = mesh.vertex_colors.active  # アクティブな頂点カラーレイヤー
                # 頂点情報をファイルに書き込む
                for vertex in vertices:
                    coord = obj.matrix_world @ vertex.co  # グローバル座標系での頂点座標
                    line = f"Vertex: {coord}, Color: "
                    if vertex_colors:
                        color = vertex_colors.data[vertex.index].color
                        line += f"({color[0]}, {color[1]}, {color[2]})"
                    else:
                        line += "(No Color)"
                    file.write(line + "\n")
    print(f"頂点情報を {output_file} に出力しました。")
else:
    print("オブジェクトが選択されていません。")

In [ ]:
import bpy

# 選択したオブジェクトの参照を取得
selected_objects = bpy.context.selected_objects
if selected_objects:
    # 出力ファイルのパスと名前
    output_file = "output.txt"
    # ファイルを開いて書き込みモードで準備
    with open(output_file, "w") as file:
        for obj in selected_objects:
            if obj.type == "MESH":
                mesh = obj.data
                vertices = mesh.vertices
                vertex_colors = mesh.vertex_colors.active  # アクティブな頂点カラーレイヤー
                # 頂点情報をファイルに書き込む
                for vertex in vertices:
                    coord = obj.matrix_world @ vertex.co  # グローバル座標系での頂点座標
                    line = ' '.join(map(str, coord))
                    if vertex_colors:
                        color = vertex_colors.data[vertex.index].color
                        line += ' ' + ' '.join(map(str, color))
                    else:
                        line += ' NoColor'
                    file.write(line + "\n")
    print(f"頂点情報を {output_file} に出力しました。")
else:
    print("オブジェクトが選択されていません。")

In [ ]:
import bpy

# Get reference to selected objects
selected_objects = bpy.context.selected_objects
if selected_objects:
    # Path and name of the output file
    output_file = "output_pineapple.txt"
    # Prepare the file by opening it in write mode
    with open(output_file, "w") as file:
        for obj in selected_objects:
            if obj.type == "MESH":
                mesh = obj.data
                vertex_colors = mesh.vertex_colors.active  # active vertex color layer
                # Write the vertex information to the file
                if vertex_colors:
                    for poly in mesh.polygons:
                        for loop_index in poly.loop_indices:
                            loop = mesh.loops[loop_index]
                            vertex = mesh.vertices[loop.vertex_index]
                            coord = obj.matrix_world @ vertex.co  # vertex coordinates in global coordinate system
                            color = vertex_colors.data[loop.index].color
                            line = ' '.join(map(str, coord)) + ' ' + ' '.join(map(str, color))
                            file.write(line + "\n")
                else:
                    for vertex in mesh.vertices:
                        coord = obj.matrix_world @ vertex.co  # vertex coordinates in global coordinate system
                        line = ' '.join(map(str, coord)) + ' NoColor'
                        file.write(line + "\n")
    print(f"Vertex information output to {output_file}.")
else:
    print("No objects are selected.")
